In [2]:
import itertools
import pandas as pd
import random
import os
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [6]:
import os, sys
from pathlib import Path

# Set up path
if os.getcwd() == '/content':
    %cd '/content/drive/MyDrive/Colab Notebooks/kaggle/Player-Contact-Detection/tutorial'
    base_path = Path('__file__').resolve().parent.parent
else:
    base_path = Path().resolve()

data_path = base_path / 'data'
submission_path = base_path / 'submission'

# Load Fully-Connected train-test dataset

In [7]:
submission = pd.read_csv(data_path / 'sample_submission.csv')

def make_filename(windows: int=0, test: bool=False, ground: bool=False):
    train_test = 'test' if test else 'train'
    type_ = 'ground' if ground else 'player'

    filename = f'{train_test}_{type_}_contact_tracking'

    if windows == 0:
        suffix = '.csv'
    elif windows >= 1:
        suffix = f'_{windows}.csv'

    return filename + suffix

def load_data(data_path, windows: int=0, ground: bool=False):
    train_filename = make_filename(windows, test=False, ground=ground)
    test_filename = make_filename(windows, test=True, ground=ground)

    try:
        train = pd.read_csv(data_path / train_filename)
        test = pd.read_csv(data_path / test_filename)

    except Exception as e:
        print("Return basic dataset.")
        load_data(data_path, 0)
        
    train.sort_values(by=['group_id', 'step'], inplace=True)
    train.reset_index(drop=True, inplace=True)
    test.sort_values(by=['group_id', 'step'], inplace=True)
    test.reset_index(drop=True, inplace=True)

    return train, test

# Imbalance Processing

In [8]:
def preprocessing(data):
    temp = data.loc[data.contact == 1]
    index = temp.index.values.tolist()

    prev_i = index[0]
    group_ids = []

    for i in index:
        if (i - prev_i) > 1:
            sr = data.loc[prev_i,:]
            group_id = sr.loc['group_id']
            group_ids.append(group_id)
        prev_i = i
    
    drop_cols = ['contact_id', 'group_id', 'game_play',
                 'step', 'nfl_player_id_1', 'nfl_player_id_2']

    result = data.loc[data.group_id.isin(group_ids), :]
    result.drop(columns=drop_cols, inplace=True)
    return result

In [9]:
def preprocessing(data):
    temp = data.loc[data.contact == 1]
    index = temp.index.values.tolist()

    prev_i = index[0]
    group_ids = []

    for i in index:
        if (i - prev_i) > 1:
            sr = data.loc[prev_i,:]
            group_id = sr.loc['group_id']
            group_ids.append(group_id)
        prev_i = i
    
    drop_cols = ['contact_id', 'group_id', 'game_play',
                 'step', 'nfl_player_id_1', 'nfl_player_id_2']

    result = data.loc[data.group_id.isin(group_ids), :]
    result.drop(columns=drop_cols, inplace=True)
    return result

# Transformer

In [10]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs

In [11]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, dff, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(dff, activation="relu"),
             tf.keras.layers.Dense(embedding_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs) # 첫번째 서브층 : 멀티 헤드 어텐션
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output) # Add & Norm
        ffn_output = self.ffn(out1) # 두번째 서브층 : 포지션 와이즈 피드 포워드 신경망
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output) # Add & Norm

In [12]:
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embedding_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.pos_emb = tf.keras.layers.Embedding(max_len, embedding_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [25]:
vocab_size = 128  # 빈도수 상위 2만개의 단어만 사용
max_len = 11  # 문장의 최대 길이

# print('훈련용 리뷰 개수 : {}'.format(len(X_train)))
# print('테스트용 리뷰 개수 : {}'.format(len(X_train)))

NameError: name 'X_train' is not defined

In [26]:
def transformer():
    embedding_dim = 32  # 각 단어의 임베딩 벡터의 차원
    num_heads = 2  # 어텐션 헤드의 수
    dff = 32  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기

    inputs = tf.keras.layers.Input(shape=(max_len,))
    embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
    x = embedding_layer(inputs)

    x = tf.keras.layers.Conv1D(filters = 64, kernel_size = 2, padding='valid', activation='relu')(x)
    x = tf.keras.layers.GlobalMaxPooling1D()(x)

    #print(x.shape)
    #transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
    transformer_block = TransformerBlock(64, num_heads, dff)
    x = transformer_block(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    #x = tf.keras.layers.GlobalMaxPooling1D()(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(32, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    outputs = tf.keras.layers.Dense(3, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    adam = tf.optimizers.Adam(learning_rate = 0.001)
    model.compile(optimizer = adam, loss = "sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [27]:
def model_result(train: pd.DataFrame, test: pd.DataFrame,
                 windows: int, save: bool=False) -> pd.DataFrame:
    drop_cols = ['contact_id', 'group_id', 'game_play',
                 'step', 'nfl_player_id_1', 'nfl_player_id_2']
    target_column = 'contact'

    X = train.drop(columns=[target_column])
    y = train.contact
    X_test = test.drop(columns=drop_cols + [target_column])

    # train, validation dataset split
    X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                      test_size=0.2,
                                                      random_state=42)

    # model initializing
    model = KerasClassifier(build_fn=transformer, epochs=90, batch_size=32, verbose=0)
    model._estimator_type="classifier"
    
    print(X_train.shape)
    # model training
    model.fit(X_train, y_train, batch_size=1024, epochs=2000)
    
    # Find optimized parameters(batch size and epochs)
    params = {
    'batch_size':[16, 32],
    'epochs':[60, 80, 90, 120, 150],
    }
    
    grid = GridSearchCV(model, param_grid=params, cv=3, refit=True)
    grid.fit(X_train, train_y)
    
    # prediction
    y_train_pred = model.predict(X_train)
    y_pred = model.predict(X_val)

    # scoring
#     score_train = clf.score(y_train, y_train_pred)
#     score_val = clf.score(y_val, y_pred)
#     print("Train %s score is %.4f" % (clf.metric_, score_train))
#     print("Validation %s score is %.4f" % (clf.metric_, score_val))
    score_train = mcc(y_train, y_train_pred)
    score_val = mcc(y_val, y_pred)
    print("Train mcc score is %.4f" % (score_train))
    print("Validation mcc score is %.4f" % (score_val))

    y_test = clf.predict(X_test)

    test.loc[:, target_column] = y_test
    return test.loc[:, ['contact_id', target_column]], score_val

# Inference

In [28]:
def inference(windows):
    train_player, test_player = load_data(data_path, windows, ground=False)
    train_ground, test_ground = load_data(data_path, windows, ground=True)

    train_player_balanced = preprocessing(train_player)
    train_ground_balanced = preprocessing(train_ground)

    print(f'#----------------- Windows: {str(windows): <2s}-------------------#')
    print('#------ Model: contact between players -----#')
    player, score_player = model_result(train_player_balanced, test_player, windows)
    print()
    print('#------ Model: contact player-ground -------#')
    ground, score_ground = model_result(train_ground_balanced, test_ground, windows)
    print('#-------------------------------------------------#')
    print()
    score_ls.append((score_player, score_ground))
    submission = pd.merge(submission.loc[:, 'contact_id'],
                          pd.concat([test_player, test_ground]),
                          on='contact_id', how='left')
    return submission

# Submission

In [29]:
def save_submission(submission, windows):
    num = 1
    try:
        for filename in os.listdir(submission_path):
            if filename.contains(f'submission_win{windows}'):
                num += 1
    except FileNotFoundError:
        num = 1

    filename = f'submission_win{windows}_ver{num}.csv'
    submission.to_csv(base_path / 'submission' / filename, index=False)

# Main.py

In [ ]:
window = 0
submission = inference(window)
submission

/var/folders/4d/_5h82f8n3pz9ll_43q6tphvw0000gn/T/ipykernel_19716/1953976632.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.drop(columns=drop_cols, inplace=True)
/var/folders/4d/_5h82f8n3pz9ll_43q6tphvw0000gn/T/ipykernel_19716/1953976632.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.drop(columns=drop_cols, inplace=True)
/var/folders/4d/_5h82f8n3pz9ll_43q6tphvw0000gn/T/ipykernel_19716/2554853103.py:17: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  

#----------------- Windows: 0 -------------------#
#------ Model: contact between players -----#
(213884, 11)


2023-02-07 19:21:34.041733: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
